In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import sys
sys.path.append("../") # go to parent dir

import numpy as np
import torch
from torch import nn, optim
import matplotlib.pyplot as plt

from itertools import product

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
np.random.seed(123)
torch.manual_seed(123)

### Step 1: Generate Data

In [5]:
from synthetic.generate import SingleTaskTreeDepsGenerator

K = 2
M = 10
N = 10000

# Generate the true class balance to be recovered
class_balance = np.ones(K)/K + np.random.random(K)/5.
class_balance /= class_balance.sum()

data = SingleTaskTreeDepsGenerator(N, M, K, class_balance, edges=[(0,1)])
print (f"LF Dependencies: {data.E}")
print (f"Class Balance: {data.p}")

LF Dependencies: [(0, 1)]
Class Balance: [0.47192333 0.52807667]


In [6]:
from metal.analysis import lf_summary
lf_summary(data.L,data.Y)

,Polarity,Coverage,Overlaps,Conflicts,Correct,Incorrect,Emp. Acc.
0,"[1, 2]",0.8958,0.8958,0.7156,7865,1093,0.877986
1,"[1, 2]",0.8937,0.8937,0.7139,7865,1072,0.880049
2,"[1, 2]",0.8657,0.8657,0.6972,7272,1385,0.840014
3,"[1, 2]",0.8609,0.8609,0.6882,7248,1361,0.841910
4,"[1, 2]",0.7973,0.7973,0.6479,5945,2028,0.745642
5,"[1, 2]",0.9077,0.9077,0.7261,8199,878,0.903272
6,"[1, 2]",0.8914,0.8914,0.7132,7876,1038,0.883554
7,"[1, 2]",0.7645,0.7645,0.6229,5385,2260,0.704382
8,"[1, 2]",0.7849,0.7849,0.6452,5665,2184,0.721748
9,"[1, 2]",0.8386,0.8386,0.6756,6720,1666,0.801336


### Step 2: Learn Dependencies using `DependencyLearnerModel`
**TEMP: hardcoding for now**

In [7]:
edges = data.E
L = np.array(data.L.todense())

### Step 3: Recover the class balance using subset of independent LFs

In [8]:
# Generate indices for independent LFs
ind_lfs = []
for i in range(M):
    if i not in list(sum(edges, ())):
        ind_lfs.append(i)
L_ind = L[:,ind_lfs]

In [9]:
%%time
from metal.label_model.class_balance import ClassBalanceModel

model = ClassBalanceModel(K, abstains=True)
model.train_model(L=L_ind, verbose=False)

print(f"Estimated class balance: {model.class_balance}")
print(f"True class balance: {class_balance}")
print()

Estimated class balance: [0.4794574 0.5209987]
True class balance: [0.47192333 0.52807667]

CPU times: user 5.8 s, sys: 80 ms, total: 5.88 s
Wall time: 1.09 s


### Step 4: Train `LabelModel` and predict training labels
**TEMP: not including constraints from Class Balance Model**

In [17]:
%%time
from metal.label_model import LabelModel
label_model = LabelModel(k=K, seed=123)

label_model.train_model(data.L, class_balance=model.class_balance.astype(float), deps=edges, lr=1e-3, n_epochs=500, log_train_every=50)

Computing O^{-1}...
Estimating Z...
[50 epo]: TRAIN:[loss=38.438]
[100 epo]: TRAIN:[loss=12.843]
[150 epo]: TRAIN:[loss=12.477]
[200 epo]: TRAIN:[loss=12.476]
[250 epo]: TRAIN:[loss=12.476]
[300 epo]: TRAIN:[loss=12.476]
[350 epo]: TRAIN:[loss=12.476]
[400 epo]: TRAIN:[loss=12.476]
[450 epo]: TRAIN:[loss=12.476]
[500 epo]: TRAIN:[loss=12.476]
Finished Training
Estimating \mu...
[50 epo]: TRAIN:[loss=0.056]
[100 epo]: TRAIN:[loss=0.001]
[150 epo]: TRAIN:[loss=0.001]
[200 epo]: TRAIN:[loss=0.000]
[250 epo]: TRAIN:[loss=0.000]
[300 epo]: TRAIN:[loss=0.000]
[350 epo]: TRAIN:[loss=0.000]
[400 epo]: TRAIN:[loss=0.000]
[450 epo]: TRAIN:[loss=0.000]
[500 epo]: TRAIN:[loss=0.000]
Finished Training
CPU times: user 13 s, sys: 2.52 s, total: 15.5 s
Wall time: 805 ms


In [18]:
print('Trained Label Model Metrics:')
scores = label_model.score((data.L, data.Y), metric=['accuracy','precision', 'recall', 'f1'])

Trained Label Model Metrics:
Accuracy: 0.984
Precision: 0.985
Recall: 0.981
F1: 0.983
        y=1    y=2   
 l=1   4665    71    
 l=2    90    5174   


### Step 4b: Compare to MajorityVote and No Class Balance+Dependencies LabelModel

**Baseline: No Class Balance or Dependencies**

In [12]:
%%time
from metal.label_model import LabelModel
label_model = LabelModel(k=K, seed=123)

label_model.train_model(data.L, n_epochs=500, log_train_every=50)

Computing O...
Estimating \mu...
[50 epo]: TRAIN:[loss=0.056]
[100 epo]: TRAIN:[loss=0.011]
[150 epo]: TRAIN:[loss=0.011]
[200 epo]: TRAIN:[loss=0.011]
[250 epo]: TRAIN:[loss=0.011]
[300 epo]: TRAIN:[loss=0.011]
[350 epo]: TRAIN:[loss=0.011]
[400 epo]: TRAIN:[loss=0.011]
[450 epo]: TRAIN:[loss=0.011]
[500 epo]: TRAIN:[loss=0.011]
Finished Training
CPU times: user 16.2 s, sys: 104 ms, total: 16.3 s
Wall time: 594 ms


In [13]:
print('Trained Label Model Metrics:')
scores = label_model.score((data.L, data.Y), metric=['accuracy','precision', 'recall', 'f1'])

Trained Label Model Metrics:
Accuracy: 0.980
Precision: 0.972
Recall: 0.986
F1: 0.979
        y=1    y=2   
 l=1   4688    133   
 l=2    67    5112   


**Baseline: Majority Vote**

In [14]:
from metal.label_model.baselines import MajorityLabelVoter

mv = MajorityLabelVoter(seed=123)
print('Majority Label Voter Metrics:')
scores = mv.score((data.L, data.Y), metric=['accuracy','precision', 'recall', 'f1'])

Majority Label Voter Metrics:
Accuracy: 0.982
Precision: 0.986
Recall: 0.976
F1: 0.981
        y=1    y=2   
 l=1   4640    66    
 l=2    115   5179   
